In [25]:
!pip install mutagen
!pip install librosa
!pip install pytables
!pip install tables


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: C:\Users\grzto\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: C:\Users\grzto\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip


ERROR: Could not find a version that satisfies the requirement pytables (from versions: none)
ERROR: No matching distribution found for pytables

[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: C:\Users\grzto\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip


  Obtaining dependency information for tables from https://files.pythonhosted.org/packages/28/67/f710befc6a57a339ac0aa7272c7df58014fff4f8863c16f683894b8c7893/tables-3.9.2-cp39-cp39-win_amd64.whl.metadata
  Obtaining dependency information for numexpr>=2.6.2 from https://files.pythonhosted.org/packages/70/81/affb9ff26e8accb210fe5585b095bd6872f5614d18b76cd53888e955ed9a/numexpr-2.10.2-cp39-cp39-win_amd64.whl.metadata
  Obtaining dependency information for py-cpuinfo from https://files.pythonhosted.org/packages/e0/a9/023730ba63db1e494a271cb018dcd361bd2c917ba7004c3e49d5daf795a2/py_cpuinfo-9.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for blosc2>=2.3.0 from https://files.pythonhosted.org/packages/5c/c1/f80115c66a181e2ff18027818d3e1f37a3d5133b350b3e25189e53e28cd8/blosc2-2.5.1-cp39-cp39-win_amd64.whl.metadata
  Obtaining dependency information for ndindex>=1.4 from https://files.pythonhosted.org/packages/ee/3d/674b0b74b65ad43149e43048520c4e556b15c05abb519721da11a12d88d0/nd


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: C:\Users\grzto\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip


In [65]:
import mutagen
import os
from pathlib import Path
import librosa
import pandas as pd
import warnings
import re

In [35]:
current_dir = Path(os.getcwd())
linkin_park = current_dir / 'previews' / 'BURN IT DOWN - Linkin Park - LIVING THINGS.mp3'

In [36]:
linkin_park_file = mutagen.File(linkin_park)
linkin_park_file.info.pprint()

'MPEG 1 layer 3, 128000 bps (CBR?), 44100 Hz, 2 chn, 29.99 seconds'

In [60]:
warnings.filterwarnings('ignore', category=UserWarning)
output_file_path = 'dataset_with_amps.parquet'

if os.path.exists(output_file_path):
    print(f"file exists {output_file_path}. deleting it.")
    os.remove(output_file_path)

clear_dataset = pd.read_csv('cleared.csv')

print(merged.dtypes)
merged = merged.astype({
    "energy": "float64",
    "tempo": "float64",
    "danceability": "float64",
    "playlist_genre": "string",
    "loudness": "float64",
    "liveness": "float64",
    "valence": "float64",
    "track_artist": "string",
    "time_signature": "int64",
    "speechiness": "float64",
    "track_popularity": "int64",
    "track_album_name": "string",
    "track_name": "string",
    "instrumentalness": "float64",
    "mode": "int64",
    "key": "int64",
    "acousticness": "float64",
    "playlist_subgenre": "string",
    "type": "string",
    "playlist_id": "string",
    "sampling_rate": "int64"
})

merged["track_album_release_date"] = pd.to_datetime(merged["track_album_release_date"], errors="coerce").dt.strftime("%Y-%m")


energy                      object
tempo                       object
danceability                object
playlist_genre              object
loudness                    object
liveness                    object
valence                     object
track_artist                object
time_signature              object
speechiness                 object
track_popularity            object
track_album_name            object
track_name                  object
track_album_release_date    object
instrumentalness            object
mode                        object
key                         object
acousticness                object
id                          object
playlist_subgenre           object
type                        object
playlist_id                 object
amplitudes                  object
sampling_rate                int64
dtype: object


In [74]:
amps = []
srs = []


for _, row in clear_dataset.iterrows():
    song_name = row["track_name"]
    artist = row["track_artist"]
    album = row["track_album_name"]

    title_mp3 = f"{song_name} - {artist} - {album}"
    title_mp3 = re.sub(r'[\\/*?:"<>|]', "_", f"{song_name} - {artist} - {album}")
    try:
        file_path = os.path.join("previews", f"{title_mp3}.mp3")
        y, sr = librosa.load(file_path, sr=None)
    except:
        song_name = re.sub(r'[\\/*?:"<>|]', "_", f"{song_name}")
        pattern = re.compile(rf"^{re.escape(song_name)}.*\.mp3$", re.IGNORECASE)
        matches = [f for f in os.listdir('previews') if pattern.match(f)]
        file_path = os.path.join('previews', matches[0])

        y, sr = librosa.load(file_path, sr=None)

    amps.append(y)
    srs.append(sr)

clear_dataset["amplitudes"] = amps
clear_dataset["sampling_rate"] = srs

clear_dataset.to_parquet(output_file_path, index=False)



WHATCHU KNO ABOUT ME (with Sexyy Red) - GloRilla, Sexyy Red - WHATCHU KNO ABOUT ME (with Sexyy Red)
WHATCHU KNO ABOUT ME (with Sexyy Red)
['WHATCHU KNO ABOUT ME (with Sexyy Red) - GloRilla - GLORIOUS.mp3']
Fortnight (feat. Post Malone) - Taylor Swift, Post Malone - THE TORTURED POETS DEPARTMENT
Fortnight (feat. Post Malone)
['Fortnight (feat. Post Malone) - Taylor Swift - THE TORTURED POETS DEPARTMENT.mp3']
Whole Lotta Love - 1990 Remaster - Led Zeppelin - Led Zeppelin II (1994 Remaster)
Whole Lotta Love - 1990 Remaster
["Whole Lotta Love - 1990 Remaster - Tina Turner - Queen Of Rock 'n' Roll.mp3"]
Psycho Killer - 2005 Remaster - Talking Heads - Talking Heads '77 (Deluxe Version)
Psycho Killer - 2005 Remaster
['Psycho Killer - 2005 Remaster - Talking Heads - The Best of Talking Heads.mp3']
Tchaikovsky_ Swan Lake, Op. 20, Act 2_ No. 10, Scene. Moderato - Pyotr Ilyich Tchaikovsky, André Previn, London Symphony Orchestra - Tchaikovsky_ Swan Lake
Tchaikovsky_ Swan Lake, Op. 20, Act 2_ No. 

KeyboardInterrupt: 